In [ ]:
from pathlib2 import Path
from tqdm.notebook import tqdm

import numpy as np
import scipy.io as spio

import matplotlib.pyplot as plt
import matplotlib.animation as animation

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams["animation.embed_limit"] = 512
plt.rcParams['figure.dpi'] = 150  
%matplotlib inline

## Load data

In [ ]:
root = Path("/media/agjvc_rad3/_TESTKOLLEKTIV/Daten")

In [ ]:
stem = np.vectorize(lambda s: s.parent.parent)
series = np.array(sorted(root.glob("*/*/Series*/dicoms.mat")))
cases = np.unique(stem(series))
print(f"Cases: {len(cases)}, Series: {len(series)}")

## Visualisation

### Series

In [ ]:
case_idx = 0
assert(case_idx < len(cases))

In [ ]:
case_series = series[stem(series) == cases[0]]
case_series_len = len(case_series)
case_series_data = [(spio.loadmat(path)["dcm"]) for path in tqdm(case_series)]
assert(case_series_len > 0)

case_sequence_len = case_series_data[0].shape[0]
print(f"Series: {case_series_len}, Sequence-length: {case_sequence_len}")

In [ ]:
case_sequence_idx = 35
assert(case_sequence_idx < case_sequence_len)

In [ ]:
fig, ax = plt.subplots(1, case_series_len, figsize=(case_series_len, 1), dpi=300)

for i, d in enumerate(case_series_data):
    ax[i].imshow(d[case_sequence_idx], interpolation="nearest", cmap="gray")
    ax[i].set_title(f"Case {case_idx} Series {i}", fontsize=4)
    ax[i].set_xticks([])
    ax[i].set_yticks([])

plt.show()
plt.close()

### Frames

In [ ]:
case_series_idx = 3
assert(case_series_idx < case_series_len)
image_data = case_series_data[case_series_idx]

In [ ]:
# %%script echo "Skip"
fig, ax = plt.subplots()

im = ax.imshow(image_data[0], animated=True, cmap="gray")
def animate(i):
    im.set_data(image_data[i])
    return [im]

animation.FuncAnimation(fig, 
                        animate,
                        frames=image_data.shape[0], 
                        blit=True)

## Analysis

In [ ]:
rows, cols = 2, 5
np.random.seed(42)
rand_idx = np.random.randint(0, len(series), rows * cols)

In [ ]:
series_data = [spio.loadmat(s)["dcm"] for s in tqdm(series[rand_idx])]

In [ ]:
def calculate_stats(image):
    image_means = image.mean(axis=(1, 2))
    image_vars = image.var(axis=(1, 2))
    return image_means, image_vars

In [ ]:
series_stats = [calculate_stats(s) for s in tqdm(series_data)]

In [ ]:
figsize=(16,5)
fig_means, ax_means = plt.subplots(rows, cols, figsize=figsize)
fig_vars, ax_vars = plt.subplots(rows, cols, figsize=figsize)

fig_means.suptitle("Mean")
fig_vars.suptitle("Variance")

fig_means.set_tight_layout(True)
fig_vars.set_tight_layout(True)

def axis_plot(ax, i, j, data, ridx):
    ax[i][j].plot(data, "-", lw=1)
    ax[i][j].set_title(f"image series {ridx}")

for i in range(rows):
    for j in range(cols):
        idx = i * j + j
        ridx = rand_idx[idx]
        im_means, im_vars = series_stats[idx]
        axis_plot(ax_means, i, j, im_means, ridx)
        axis_plot(ax_vars, i, j, im_vars, ridx)

plt.show()
plt.close()
